In [1]:
import pandas as pd
import numpy as np
import requests
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

## Stations Data

In [3]:
data=pd.read_csv("/Users/jchen056/Data/hourly_data_aggregated.csv")
data.drop(columns=['Unnamed: 0'],axis=0,inplace=True)
data.head()

,station_complex,borough,date,hour,month,weekdays,num_lines,ridership,TMAX,TMIN,PRCP,avg_T
0,1 Av (L),M,2022-02-01,0,2,1,1,124,36,23,0.0,29.5
1,1 Av (L),M,2022-02-01,1,2,1,1,28,36,23,0.0,29.5
2,1 Av (L),M,2022-02-01,2,2,1,1,38,36,23,0.0,29.5
3,1 Av (L),M,2022-02-01,3,2,1,1,14,36,23,0.0,29.5
4,1 Av (L),M,2022-02-01,4,2,1,1,26,36,23,0.0,29.5


In [50]:
data['station_complex'].unique()

array(['1 Av (L)', '103 St (1)', '103 St (6)', '103 St (B,C)',
       '103 St-Corona Plaza (7)', '104 St (A)', '104 St (J,Z)',
       '110 St (6)', '111 St (7)', '111 St (A)', '111 St (J)',
       '116 St (2,3)', '116 St (6)', '116 St (B,C)',
       '116 St-Columbia University (1)', '121 St (J,Z)', '125 St (1)',
       '125 St (2,3)', '125 St (4,5,6)', '125 St (A,B,C,D)',
       '135 St (2,3)', '135 St (B,C)', '137 St-City College (1)',
       '138 St-Grand Concourse (4,5)', '14 St (A,C,E)/8 Av (L)',
       '14 St (F,M,1,2,3)/6 Av (L)', '14 St-Union Sq (L,N,Q,R,W,4,5,6)',
       '145 St (1)', '145 St (3)', '145 St (A,B,C,D)',
       '149 St-Grand Concourse (2,4,5)', '15 St-Prospect Park (F,G)',
       '155 St (B,D)', '155 St (C)', '157 St (1)',
       '161 St-Yankee Stadium (B,D,4)', '163 St-Amsterdam Av (C)',
       '167 St (4)', '167 St (B,D)', '168 St (A,C,1)', '169 St (F)',
       '170 St (4)', '170 St (B,D)', '174 St (2,5)', '174-175 Sts (B,D)',
       '175 St (A)', '176 St (4)', 

In [51]:
stationA=data[data['station_complex']=='Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)/Bryant Pk (B,D,F,M)/5 Av (7)']
stationA.head(2)

,station_complex,borough,date,hour,month,weekdays,num_lines,ridership,TMAX,TMIN,PRCP,avg_T
5699464,"Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,...",M,2022-02-01,0,2,1,16,1578,36,23,0.0,29.5
5699465,"Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,...",M,2022-02-01,1,2,1,16,456,36,23,0.0,29.5


In [52]:
stationA_filtered=stationA[['hour', 'month', 'weekdays','ridership','PRCP', 'avg_T']].copy()
stationA_filtered.head()

,hour,month,weekdays,ridership,PRCP,avg_T
5699464,0,2,1,1578,0.0,29.5
5699465,1,2,1,456,0.0,29.5
5699466,2,2,1,278,0.0,29.5
5699467,3,2,1,172,0.0,29.5
5699468,4,2,1,352,0.0,29.5


In [53]:
data_w_dummies=pd.get_dummies(stationA_filtered,
                              columns=['hour','month','weekdays'],
                             drop_first=True)
data_w_dummies.head()

,ridership,PRCP,avg_T,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,...,month_9,month_10,month_11,month_12,weekdays_1,weekdays_2,weekdays_3,weekdays_4,weekdays_5,weekdays_6
5699464,1578,0.0,29.5,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5699465,456,0.0,29.5,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5699466,278,0.0,29.5,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5699467,172,0.0,29.5,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5699468,352,0.0,29.5,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [54]:
y=data_w_dummies['ridership']
x=data_w_dummies[[i for i in data_w_dummies.columns if i!='ridership']]
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, random_state=42)

In [55]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
poly_regression=LinearRegression()
poly=PolynomialFeatures(degree=2)
X_train_modified=poly.fit_transform(X_train)

poly_regression.fit(X_train_modified,y_train)

LinearRegression()

# Dynamic Pricing Model+Clustering to prevent Price arbitrage




# Eventbrite API

In [ ]:
#api_key_eventbrite='TBYNJXQW5UVLWP4JPUL6'

In [ ]:
import requests

def get_large_events_ticketmaster(city, api_key):
    """
    Fetch large events from Ticketmaster API for a specific city.
    """
    url = "https://app.ticketmaster.com/discovery/v2/events.json"
    params = {
        "apikey": api_key,
        "city": city,
        "size": "large"
    }
    response = requests.get(url, params=params)
    print(response)
    events = response.json()

    large_events = []
    for event in events.get("events", []):
        venue_id = event.get("venue_id")
        venue_details = get_venue_details(venue_id, api_key)

        if venue_details.get("capacity") > 5000:
            large_events.append(event)

    return large_events


#events = get_large_events_ticketmaster('New York', 'your_api_key')
print(events)


[]


In [ ]:
import tweepy

def analyze_social_media_for_events(api_key, api_secret_key, access_token, access_token_secret, city, keywords):
    auth = tweepy.OAuthHandler(api_key, api_secret_key)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)
    tweets = api.search_tweets(q=f"{keywords} + near:\"{city}\"", count=100)
    events = [{'tweet': tweet.text, 'user': tweet.user.screen_name} for tweet in tweets]
    return events

# with keywords: events
events = analyze_social_media_for_events('your_api_key', 'your_api_secret_key',
                                         'your_access_token', 'your_access_token_secret',
                                         'New York', '#event OR #festival on #protest')
for event in events:
    print(event)


In [ ]:
#Problem : tweets limit 1500/mo , (maybe use multiple API KEYS)

##Dynamic Pricing with Sigmoid adjustment

In [ ]:

    #event_factor = get_event_factor(date_time, station_id) #from mta complaints
    #traffic_factor = get_traffic_factor(date_time, station_id)
    #cluster_factor = get_cluster_factor(station_id)




In [25]:
import numpy as np
import math

# Example sigmoid function for fare adjustment
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def calculate_dynamic_fare_with_sigmoid(input_data, base_fare, max_capacity):

    traffic_factor=1
    cluster_factor=1
    event_factor=1

    sigmoid_midpoint = max_capacity / 2  # Midpoint of the sigmoid function
    sigmoid_steepness = 0.0001  # Controls how steep the sigmoid curve small for more gradual increase.

    predicted_ridership=round(poly_regression.predict(poly.fit_transform(input_data.values))[0],2)
    
    # Normalize ridership value for sigmoid function
    print(predicted_ridership)
    normalized_ridership = (predicted_ridership - sigmoid_midpoint) * sigmoid_steepness
    print(normalized_ridership)
    sigmoid_adjustment = sigmoid(normalized_ridership)

    dynamic_fare = base_fare * (1 + sigmoid_adjustment) * event_factor * traffic_factor * cluster_factor

    return np.clip(dynamic_fare, 0,30)

In [63]:
input_data=pd.DataFrame(np.zeros((1,42)),
                    columns=['PRCP', 'avg_T', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5',
    'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12',
    'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18',
    'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'month_2',
    'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8',
    'month_9', 'month_10', 'month_11', 'month_12', 'weekdays_1',
    'weekdays_2', 'weekdays_3', 'weekdays_4', 'weekdays_5', 'weekdays_6'])
input_data.loc[0,'PRCP']=0
input_data.loc[0,'avg_T']=70
input_data.loc[0,'hour_12']=1
input_data.loc[0,'month_5']=1
input_data.loc[0,'weekdays_6']=1

In [64]:
# Example usage
base_fare = 2  # Base fare
max_capacity = 2000 #https://www.nycsubway.org/wiki/Loading_Speed_A_Major_Factor_in_Design_of_New_York_Subway_Cars_(1931)

fare = calculate_dynamic_fare_with_sigmoid(input_data, base_fare, max_capacity)
print(f"Dynamic Fare: ${fare:.4f}")

4898.74
0.389874
Dynamic Fare: $3.1925


## Cluster based Pricing

